# New Section

In [19]:
# Upload your CSV file using the Colab file upload widget
from google.colab import files
uploaded = files.upload()

# Import necessary libraries
import io
import pandas as pd
import folium
import requests

# Load your sales data from the uploaded file (adjust the file name if necessary)
# This assumes your CSV file has a 'Country' and a 'Sales' column.
for filename in uploaded.keys():
    sales_df = pd.read_csv(io.BytesIO(uploaded[filename]))
    print(f'Loaded {filename} with shape: {sales_df.shape}')

# Define the list of MENA region countries
mena_countries = [
    'Algeria', 'Bahrain', 'Egypt', 'Iran', 'Iraq', 'Israel', 'Jordan',
    'Kuwait', 'Lebanon', 'Libya', 'Morocco', 'Oman', 'Qatar', 'Saudi Arabia',
    'Syria', 'Tunisia', 'United Arab Emirates', 'Yemen', 'Palestine'
]

# Filter the data to include only MENA countries
sales_mena = sales_df[sales_df['Country'].isin(mena_countries)]

# Load world country boundaries as a GeoJSON
geojson_url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data/world-countries.json'
geo_data = requests.get(geojson_url).json()

# Filter the GeoJSON to include only MENA countries
mena_geo = {
    "type": "FeatureCollection",
    "features": [feature for feature in geo_data["features"] if feature["properties"]["name"] in mena_countries]
}

# Create a Folium map centered on the MENA region
m = folium.Map(location=[26, 17], zoom_start=5)

# Add the choropleth layer with a green color scale and clear country borders
folium.Choropleth(
    geo_data=mena_geo,
    name='choropleth',
    data=sales_mena,
    columns=['Country', 'Sales'],
    key_on='feature.properties.name',
    fill_color='YlGn',         # Green color scale: lighter greens for lower sales, darker for higher
    fill_opacity=0.7,
    line_opacity=1,
    line_color='black',        # Clear black borders between countries
    legend_name='Sales (in USD)'
).add_to(m)

# Optionally, add tooltips to display country names when hovering over them
folium.GeoJson(
    mena_geo,
    style_function=lambda feature: {
        'fillColor': 'transparent',
        'color': 'black',
        'weight': 2,
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['name'],
        aliases=['Country:'],
    )
).add_to(m)

# Add a custom glossary box next to the map
legend_html = '''
<div style="position: fixed;
     bottom: 50px; left: 50px; width: 250px; height: 120px;
     border:2px solid grey; z-index:9999; font-size:14px;
     background-color:white;
     padding: 10px;
     ">
     <b>Glossary</b><br>
     <i>Sales Scale:</i><br>
     Lighter green: Lower sales<br>
     Darker green: Higher sales<br>
     (Values in USD)
</div>
'''
m.get_root().html.add_child(folium.Element(legend_html))

# Display the interactive map
m


Saving 2024_sales_per_country.csv to 2024_sales_per_country.csv
Loaded 2024_sales_per_country.csv with shape: (8, 3)


In [20]:
# Upload your CSV file using the Colab file upload widget
from google.colab import files
uploaded = files.upload()

import io
import pandas as pd
import folium
import requests

# Load your sales data from the uploaded file (adjust the file name if necessary)
for filename in uploaded.keys():
    sales_df = pd.read_csv(io.BytesIO(uploaded[filename]))
    print(f'Loaded {filename} with shape: {sales_df.shape}')

# Define list of MENA countries (for reference) and mapping to their ISO codes
mena_countries = [
    'Algeria', 'Bahrain', 'Egypt', 'Iran', 'Iraq', 'Israel', 'Jordan',
    'Kuwait', 'Lebanon', 'Libya', 'Morocco', 'Oman', 'Qatar', 'Saudi Arabia',
    'Syria', 'Tunisia', 'United Arab Emirates', 'Yemen', 'Palestine'
]

country_to_iso = {
    'Algeria': 'DZA',
    'Bahrain': 'BHR',
    'Egypt': 'EGY',
    'Iran': 'IRN',
    'Iraq': 'IRQ',
    'Israel': 'ISR',
    'Jordan': 'JOR',
    'Kuwait': 'KWT',
    'Lebanon': 'LBN',
    'Libya': 'LBY',
    'Morocco': 'MAR',
    'Oman': 'OMN',
    'Qatar': 'QAT',
    'Saudi Arabia': 'SAU',
    'Syria': 'SYR',
    'Tunisia': 'TUN',
    'United Arab Emirates': 'ARE',
    'Yemen': 'YEM',
    'Palestine': 'PSE'
}

# Filter the data to include only MENA countries and work on a copy
sales_mena = sales_df[sales_df['Country'].isin(mena_countries)].copy()

# Convert the Sales column to numeric (if not already)
sales_mena['Sales'] = pd.to_numeric(sales_mena['Sales'], errors='coerce')

# Map the country names to their ISO codes
sales_mena['ISO'] = sales_mena['Country'].map(country_to_iso)

# Get the list of ISO codes from your data (drop any missing values)
mena_iso_codes = sales_mena['ISO'].dropna().unique().tolist()

# Load world country boundaries as a GeoJSON
geojson_url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data/world-countries.json'
geo_data = requests.get(geojson_url).json()

# Filter the GeoJSON to include only the features corresponding to your MENA ISO codes
mena_geo = {
    "type": "FeatureCollection",
    "features": [feature for feature in geo_data["features"] if feature["id"] in mena_iso_codes]
}

# Create a Folium map centered on the MENA region
m = folium.Map(location=[26, 17], zoom_start=5)

# Add the choropleth layer with a green color scale and clear borders
folium.Choropleth(
    geo_data=mena_geo,
    name='choropleth',
    data=sales_mena,
    columns=['ISO', 'Sales'],
    key_on='feature.id',       # Match using ISO codes
    fill_color='YlGn',         # Green color scale: lighter greens for lower, darker for higher sales
    fill_opacity=0.7,
    line_opacity=1,
    line_color='black',        # Clear borders between countries
    legend_name='Sales (in USD)'
).add_to(m)

# Optionally, add tooltips to display country names when hovering
folium.GeoJson(
    mena_geo,
    style_function=lambda feature: {
        'fillColor': 'transparent',
        'color': 'black',
        'weight': 2,
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['name'],
        aliases=['Country:'],
    )
).add_to(m)

# Add a custom glossary box next to the map
legend_html = '''
<div style="position: fixed;
     bottom: 50px; left: 50px; width: 250px; height: 120px;
     border:2px solid grey; z-index:9999; font-size:14px;
     background-color:white;
     padding: 10px;
     ">
     <b>Glossary</b><br>
     <i>Sales Scale:</i><br>
     Lighter green: Lower sales<br>
     Darker green: Higher sales<br>
     (Values in USD)
</div>
'''
m.get_root().html.add_child(folium.Element(legend_html))

# Display the interactive map
m


Saving 2024_sales_per_country.csv to 2024_sales_per_country (1).csv
Loaded 2024_sales_per_country (1).csv with shape: (8, 3)


In [21]:
# Upload your CSV file using the Colab file upload widget
from google.colab import files
uploaded = files.upload()

import io
import pandas as pd
import folium
import requests

# Load your CSV file (adjust the filename if necessary)
sales_df = pd.read_csv('/mnt/data/2024_sales_per_country.csv')

# Standardize column names: remove extra spaces and title-case them
sales_df.columns = [col.strip().title() for col in sales_df.columns]

# Print out the unique country names so you can verify they match the mapping keys
print("Unique country names in data:", sales_df['Country'].unique())

# Define a mapping from your CSV's country names to ISO3 codes (as used by the GeoJSON)
country_to_iso = {
    'Algeria': 'DZA',
    'Bahrain': 'BHR',
    'Egypt': 'EGY',
    'Iran': 'IRN',
    'Iraq': 'IRQ',
    'Israel': 'ISR',
    'Jordan': 'JOR',
    'Kuwait': 'KWT',
    'Lebanon': 'LBN',  # Ensure your CSV uses "Lebanon"; adjust if necessary.
    'Libya': 'LBY',
    'Morocco': 'MAR',
    'Oman': 'OMN',
    'Qatar': 'QAT',
    'Saudi Arabia': 'SAU',
    'Syria': 'SYR',
    'Tunisia': 'TUN',
    'United Arab Emirates': 'ARE',
    'Uae': 'ARE',  # In case your CSV abbreviates it this way.
    'Yemen': 'YEM',
    'Palestine': 'PSE',
    'Palestinian Territories': 'PSE'  # Use this if your file uses an alternate naming.
}

# Filter the DataFrame to include only rows with countries in our mapping dictionary
sales_df = sales_df[sales_df['Country'].isin(country_to_iso.keys())].copy()

# Convert the Sales column to a numeric type (forcing non-numeric to NaN)
sales_df['Sales'] = pd.to_numeric(sales_df['Sales'], errors='coerce')

# Create a new column with the ISO codes based on our mapping
sales_df['ISO'] = sales_df['Country'].map(country_to_iso)

# Get the unique list of ISO codes present in your data
mena_iso_codes = sales_df['ISO'].dropna().unique().tolist()

# Load a GeoJSON file containing world country boundaries (the 'id' field here is the ISO3 code)
geojson_url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data/world-countries.json'
geo_data = requests.get(geojson_url).json()

# Filter the GeoJSON to include only those features with IDs matching your MENA ISO codes
mena_geo = {
    "type": "FeatureCollection",
    "features": [feature for feature in geo_data["features"] if feature["id"] in mena_iso_codes]
}

# Create a Folium map centered on the MENA region
m = folium.Map(location=[26, 17], zoom_start=5)

# Add a choropleth layer with the green color scale and clear black borders
folium.Choropleth(
    geo_data=mena_geo,
    name='choropleth',
    data=sales_df,
    columns=['ISO', 'Sales'],
    key_on='feature.id',       # Match using ISO codes
    fill_color='YlGn',         # Uses a green color scale (lighter = lower, darker = higher sales)
    fill_opacity=0.7,
    line_opacity=1,
    line_color='black',        # Clear borders between countries
    legend_name='Sales (in USD)'
).add_to(m)

# Optionally, add tooltips so that hovering over a country displays its name
folium.GeoJson(
    mena_geo,
    style_function=lambda feature: {
        'fillColor': 'transparent',
        'color': 'black',
        'weight': 2,
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['name'],
        aliases=['Country:'],
    )
).add_to(m)

# Add a custom glossary box next to the map to explain the color scale
legend_html = '''
<div style="position: fixed;
     bottom: 50px; left: 50px; width: 250px; height: 120px;
     border:2px solid grey; z-index:9999; font-size:14px;
     background-color:white;
     padding: 10px;
     ">
     <b>Glossary</b><br>
     <i>Sales Scale:</i><br>
     Lighter green: Lower sales<br>
     Darker green: Higher sales<br>
     (Values in USD)
</div>
'''
m.get_root().html.add_child(folium.Element(legend_html))

# Display the interactive map
m


Saving 2024_sales_per_country.csv to 2024_sales_per_country (2).csv


FileNotFoundError: [Errno 2] No such file or directory: '/mnt/data/2024_sales_per_country.csv'

In [22]:
# Upload your CSV file using the Colab file upload widget
from google.colab import files
uploaded = files.upload()

import io
import pandas as pd
import folium
import requests

# Load your CSV file from the uploaded file(s)
# This assumes you'll upload only one file.
filename = list(uploaded.keys())[0]
sales_df = pd.read_csv(io.BytesIO(uploaded[filename]))
print(f'Loaded {filename} with shape: {sales_df.shape}')

# Standardize column names (remove extra spaces and title-case them)
sales_df.columns = [col.strip().title() for col in sales_df.columns]

# Print out the unique country names so you can verify they match the mapping keys
print("Unique country names in data:", sales_df['Country'].unique())

# Define a mapping from your CSV's country names to ISO3 codes (as used by the GeoJSON)
country_to_iso = {
    'Algeria': 'DZA',
    'Bahrain': 'BHR',
    'Egypt': 'EGY',
    'Iran': 'IRN',
    'Iraq': 'IRQ',
    'Israel': 'ISR',
    'Jordan': 'JOR',
    'Kuwait': 'KWT',
    'Lebanon': 'LBN',
    'Libya': 'LBY',
    'Morocco': 'MAR',
    'Oman': 'OMN',
    'Qatar': 'QAT',
    'Saudi Arabia': 'SAU',
    'Syria': 'SYR',
    'Tunisia': 'TUN',
    'United Arab Emirates': 'ARE',
    'Uae': 'ARE',  # in case your CSV uses this abbreviation
    'Yemen': 'YEM',
    'Palestine': 'PSE',
    'Palestinian Territories': 'PSE'
}

# Filter the DataFrame to include only rows with countries in our mapping dictionary
sales_df = sales_df[sales_df['Country'].isin(country_to_iso.keys())].copy()

# Convert the Sales column to numeric (forcing non-numeric to NaN)
sales_df['Sales'] = pd.to_numeric(sales_df['Sales'], errors='coerce')

# Create a new column with the ISO codes based on our mapping
sales_df['ISO'] = sales_df['Country'].map(country_to_iso)

# Get the unique list of ISO codes present in your data
mena_iso_codes = sales_df['ISO'].dropna().unique().tolist()

# Load a GeoJSON file containing world country boundaries (the 'id' field here is the ISO3 code)
geojson_url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data/world-countries.json'
geo_data = requests.get(geojson_url).json()

# Filter the GeoJSON to include only features with IDs matching your MENA ISO codes
mena_geo = {
    "type": "FeatureCollection",
    "features": [feature for feature in geo_data["features"] if feature["id"] in mena_iso_codes]
}

# Create a Folium map centered on the MENA region
m = folium.Map(location=[26, 17], zoom_start=5)

# Add the choropleth layer with the green color scale and clear black borders
folium.Choropleth(
    geo_data=mena_geo,
    name='choropleth',
    data=sales_df,
    columns=['ISO', 'Sales'],
    key_on='feature.id',       # Match using ISO codes
    fill_color='YlGn',         # Green


SyntaxError: incomplete input (<ipython-input-22-47da08bf4d42>, line 79)

In [23]:
# Upload your CSV file using the Colab file upload widget
from google.colab import files
uploaded = files.upload()

import io
import pandas as pd
import folium
import requests

# Load your CSV file from the uploaded file(s)
# This assumes you'll upload only one file.
filename = list(uploaded.keys())[0]
sales_df = pd.read_csv(io.BytesIO(uploaded[filename]))
print(f'Loaded {filename} with shape: {sales_df.shape}')

# Standardize column names (remove extra spaces and title-case them)
sales_df.columns = [col.strip().title() for col in sales_df.columns]

# Print out the unique country names so you can verify they match the mapping keys
print("Unique country names in data:", sales_df['Country'].unique())

# Define a mapping from your CSV's country names to ISO3 codes (as used by the GeoJSON)
country_to_iso = {
    'Algeria': 'DZA',
    'Bahrain': 'BHR',
    'Egypt': 'EGY',
    'Iran': 'IRN',
    'Iraq': 'IRQ',
    'Israel': 'ISR',
    'Jordan': 'JOR',
    'Kuwait': 'KWT',
    'Lebanon': 'LBN',
    'Libya': 'LBY',
    'Morocco': 'MAR',
    'Oman': 'OMN',
    'Qatar': 'QAT',
    'Saudi Arabia': 'SAU',
    'Syria': 'SYR',
    'Tunisia': 'TUN',
    'United Arab Emirates': 'ARE',
    'Uae': 'ARE',  # in case your CSV uses this abbreviation
    'Yemen': 'YEM',
    'Palestine': 'PSE',
    'Palestinian Territories': 'PSE'
}

# Filter the DataFrame to include only rows with countries in our mapping dictionary
sales_df = sales_df[sales_df['Country'].isin(country_to_iso.keys())].copy()

# Convert the Sales column to numeric (forcing non-numeric to NaN)
sales_df['Sales'] = pd.to_numeric(sales_df['Sales'], errors='coerce')

# Create a new column with the ISO codes based on our mapping
sales_df['ISO'] = sales_df['Country'].map(country_to_iso)

# Get the unique list of ISO codes present in your data
mena_iso_codes = sales_df['ISO'].dropna().unique().tolist()

# Load a GeoJSON file containing world country boundaries (the 'id' field here is the ISO3 code)
geojson_url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data/world-countries.json'
geo_data = requests.get(geojson_url).json()

# Filter the GeoJSON to include only features with IDs matching your MENA ISO codes
mena_geo = {
    "type": "FeatureCollection",
    "features": [feature for feature in geo_data["features"] if feature["id"] in mena_iso_codes]
}

# Create a Folium map centered on the MENA region
m = folium.Map(location=[26, 17], zoom_start=5)

# Add the choropleth layer with the green color scale and clear black borders
folium.Choropleth(
    geo_data=mena_geo,
    name='choropleth',
    data=sales_df,
    columns=['ISO', 'Sales'],
    key_on='feature.id',       # Match using ISO codes
    fill_color='YlGn',         # Green color scale: lighter greens for lower, darker for higher sales
    fill_opacity=0.7,
    line_opacity=1,
    line_color='black',        # Clear borders between countries
    legend_name='Sales (in USD)'
).add_to(m)

# Optionally, add tooltips to display country names when hovering
folium.GeoJson(
    mena_geo,
    style_function=lambda feature: {
        'fillColor': 'transparent',
        'color': 'black',
        'weight': 2,
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['name'],
        aliases=['Country:'],
    )
).add_to(m)

# Add a custom glossary box next to the map to explain the color scale
legend_html = '''
<div style="position: fixed;
     bottom: 50px; left: 50px; width: 250px; height: 120px;
     border:2px solid grey; z-index:9999; font-size:14px;
     background-color:white;
     padding: 10px;
     ">
     <b>Glossary</b><br>
     <i>Sales Scale:</i><br>
     Lighter green: Lower sales<br>
     Darker green: Higher sales<br>
     (Values in USD)
</div>
'''
m.get_root().html.add_child(folium.Element(legend_html))

# Display the interactive map
m


Saving 2024_sales_per_country.csv to 2024_sales_per_country (3).csv
Loaded 2024_sales_per_country (3).csv with shape: (8, 3)
Unique country names in data: ['Kuwait' 'Qatar' 'KSA ' 'Jordan ' 'Bahrain ' 'Palestine ' 'Oman']


In [24]:
# Upload your CSV file using the Colab file upload widget
from google.colab import files
uploaded = files.upload()

import io
import pandas as pd
import folium
import requests

# Load your CSV file from the uploaded file(s)
# This assumes you'll upload only one file.
filename = list(uploaded.keys())[0]
sales_df = pd.read_csv(io.BytesIO(uploaded[filename]))
print(f'Loaded {filename} with shape: {sales_df.shape}')

# Standardize column names (remove extra spaces and title-case them)
sales_df.columns = [col.strip().title() for col in sales_df.columns]

# Print out the unique country names so you can verify they match the mapping keys
print("Unique country names in data:", sales_df['Country'].unique())

# Define a mapping from your CSV's country names to ISO3 codes (as used by the GeoJSON)
country_to_iso = {
    'Algeria': 'DZA',
    'Bahrain': 'BHR',
    'Egypt': 'EGY',
    'Iran': 'IRN',
    'Iraq': 'IRQ',
    'Israel': 'ISR',
    'Jordan': 'JOR',
    'Kuwait': 'KWT',
    'Lebanon': 'LBN',
    'Libya': 'LBY',
    'Morocco': 'MAR',
    'Oman': 'OMN',
    'Qatar': 'QAT',
    'Saudi Arabia': 'SAU',
    'Syria': 'SYR',
    'Tunisia': 'TUN',
    'United Arab Emirates': 'ARE',
    'Uae': 'ARE',  # in case your CSV uses this abbreviation
    'Yemen': 'YEM',
    'Palestine': 'PSE',
    'Palestinian Territories': 'PSE'
}

# Filter the DataFrame to include only rows with countries in our mapping dictionary
sales_df = sales_df[sales_df['Country'].isin(country_to_iso.keys())].copy()

# Convert the Sales column to numeric (forcing non-numeric to NaN)
sales_df['Sales'] = pd.to_numeric(sales_df['Sales'], errors='coerce')

# Create a new column with the ISO codes based on our mapping
sales_df['ISO'] = sales_df['Country'].map(country_to_iso)

# Get the unique list of ISO codes present in your data
mena_iso_codes = sales_df['ISO'].dropna().unique().tolist()

# Load a GeoJSON file containing world country boundaries (the 'id' field here is the ISO3 code)
geojson_url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data/world-countries.json'
geo_data = requests.get(geojson_url).json()

# Filter the GeoJSON to include only features with IDs matching your MENA ISO codes
mena_geo = {
    "type": "FeatureCollection",
    "features": [feature for feature in geo_data["features"] if feature["id"] in mena_iso_codes]
}

# Create a Folium map centered on the MENA region
m = folium.Map(location=[26, 17], zoom_start=5)

# Add the choropleth layer with the green color scale and clear black borders
folium.Choropleth(
    geo_data=mena_geo,
    name='choropleth',
    data=sales_df,
    columns=['ISO', 'Sales'],
    key_on='feature.id',       # Match using ISO codes
    fill_color='YlGn',         # Green color scale: lighter greens for lower, darker for higher sales
    fill_opacity=0.7,
    line_opacity=1,
    line_color='black',        # Clear borders between countries
    legend_name='Sales (in USD)'
).add_to(m)

# Optionally, add tooltips to display country names when hovering
folium.GeoJson(
    mena_geo,
    style_function=lambda feature: {
        'fillColor': 'transparent',
        'color': 'black',
        'weight': 2,
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['name'],
        aliases=['Country:'],
    )
).add_to(m)

# Add a custom glossary box next to the map to explain the color scale
legend_html = '''
<div style="position: fixed;
     bottom: 50px; left: 50px; width: 250px; height: 120px;
     border:2px solid grey; z-index:9999; font-size:14px;
     background-color:white;
     padding: 10px;
     ">
     <b>Glossary</b><br>
     <i>Sales Scale:</i><br>
     Lighter green: Lower sales<br>
     Darker green: Higher sales<br>
     (Values in USD)
</div>
'''
m.get_root().html.add_child(folium.Element(legend_html))

# Display the interactive map
m


Saving 2024_sales_per_country.csv to 2024_sales_per_country (4).csv
Loaded 2024_sales_per_country (4).csv with shape: (8, 3)
Unique country names in data: ['Kuwait' 'Qatar' 'Saudi Arabia' 'Jordan' 'Bahrain ' 'Palestine ' 'Oman']


In [26]:
# Upload your CSV file using the Colab file upload widget
from google.colab import files
uploaded = files.upload()

import io
import pandas as pd
import folium
import requests

# Load your CSV file from the uploaded file(s)
filename = list(uploaded.keys())[0]
sales_df = pd.read_csv(io.BytesIO(uploaded[filename]))
print(f'Loaded {filename} with shape: {sales_df.shape}')

# Standardize column names: remove extra spaces and title-case them
sales_df.columns = [col.strip().title() for col in sales_df.columns]
print("Unique country names in data:", sales_df['Country'].unique())

# Define a mapping from your CSV's country names to ISO3 codes (as used by the GeoJSON)
country_to_iso = {
    'Algeria': 'DZA',
    'Bahrain': 'BHR',
    'Egypt': 'EGY',
    'Iran': 'IRN',
    'Iraq': 'IRQ',
    'Israel': 'ISR',
    'Jordan': 'JOR',
    'Kuwait': 'KWT',
    'Lebanon': 'LBN',
    'Libya': 'LBY',
    'Morocco': 'MAR',
    'Oman': 'OMN',
    'Qatar': 'QAT',
    'Saudi Arabia': 'SAU',
    'Syria': 'SYR',
    'Tunisia': 'TUN',
    'United Arab Emirates': 'ARE',
    'Uae': 'ARE',  # in case your CSV uses this abbreviation
    'Yemen': 'YEM',
    'Palestine': 'PSE',
    'Palestinian Territories': 'PSE'
}

# Filter the DataFrame to include only rows with countries in our mapping
sales_df = sales_df[sales_df['Country'].isin(country_to_iso.keys())].copy()

# Convert the Sales column to numeric (forcing non-numeric to NaN)
sales_df['Sales'] = pd.to_numeric(sales_df['Sales'], errors='coerce')

# Map country names to their ISO codes and add as a new column
sales_df['ISO'] = sales_df['Country'].map(country_to_iso)
mena_iso_codes = sales_df['ISO'].dropna().unique().tolist()

# Load a GeoJSON file containing world country boundaries (the 'id' field here is the ISO3 code)
geojson_url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data/world-countries.json'
geo_data = requests.get(geojson_url).json()

# Filter the GeoJSON to include only features with IDs matching your MENA ISO codes
mena_geo = {
    "type": "FeatureCollection",
    "features": [feature for feature in geo_data["features"] if feature["id"] in mena_iso_codes]
}

# Enrich each GeoJSON feature with Sales and Agent data from the DataFrame
for feature in mena_geo["features"]:
    iso = feature["id"]
    matching = sales_df[sales_df["ISO"] == iso]
    if not matching.empty:
        feature["properties"]["Sales"] = matching.iloc[0]["Sales"]
        # Check for an 'Agent' column; if not, check for 'Agent Name'
        if "Agent" in sales_df.columns:
            feature["properties"]["Agent"] = matching.iloc[0]["Agent"]
        elif "Agent Name" in sales_df.columns:
            feature["properties"]["Agent"] = matching.iloc[0]["Agent Name"]

# Create a Folium map centered on the MENA region
# Set max_bounds=True to restrict panning outside of the displayed region
m = folium.Map(location=[26, 17], zoom_start=5, max_bounds=True)

# Fit the map to an approximate MENA bounding box (adjust these coordinates as needed)
m.fit_bounds([[10, -10], [40, 60]])

# Add the choropleth layer with the green color scale and clear black borders
folium.Choropleth(
    geo_data=mena_geo,
    name='choropleth',
    data=sales_df,
    columns=['ISO', 'Sales'],
    key_on='feature.id',
    fill_color='YlGn',       # Green color scale: lighter for lower sales, darker for higher
    fill_opacity=0.7,
    line_opacity=1,
    line_color='black',
    legend_name='Sales (in USD)'
).add_to(m)

# Create a tooltip that displays the country name, sales, and agent
tooltip = folium.GeoJsonTooltip(
    fields=['name', 'Sales', 'Agent'],
    aliases=['Country:', 'Sales (USD):', 'Agent:'],
    localize=True
)

# Add the enriched GeoJSON layer with the custom tooltip
folium.GeoJson(
    mena_geo,
    style_function=lambda feature: {
        'fillColor': 'transparent',
        'color': 'black',
        'weight': 2,
    },
    tooltip=tooltip
).add_to(m)

# Add a custom glossary box explaining the color scale
legend_html = '''
<div style="position: fixed;
     bottom: 50px; left: 50px; width: 250px; height: 120px;
     border:2px solid grey; z-index:9999; font-size:14px;
     background-color:white;
     padding: 10px;
     ">
     <b>Glossary</b><br>
     <i>Sales Scale:</i><br>
     Lighter green: Lower sales<br>
     Darker green: Higher sales<br>
     (Values in USD)
</div>
'''
m.get_root().html.add_child(folium.Element(legend_html))

# Display the interactive map
m


Saving 2024_sales_per_country.csv to 2024_sales_per_country (6).csv
Loaded 2024_sales_per_country (6).csv with shape: (7, 3)
Unique country names in data: ['Kuwait' 'Qatar' 'Saudi Arabia' 'Jordan' 'Bahrain'
 'Palestinian Territories' 'Oman']


TypeError: Object of type int64 is not JSON serializable

In [27]:
# Upload your CSV file using the Colab file upload widget
from google.colab import files
uploaded = files.upload()

import io
import pandas as pd
import folium
import requests

# Load your CSV file from the uploaded file(s)
filename = list(uploaded.keys())[0]
sales_df = pd.read_csv(io.BytesIO(uploaded[filename]))
print(f'Loaded {filename} with shape: {sales_df.shape}')

# Standardize column names: remove extra spaces and title-case them
sales_df.columns = [col.strip().title() for col in sales_df.columns]
print("Unique country names in data:", sales_df['Country'].unique())

# Define a mapping from your CSV's country names to ISO3 codes (as used by the GeoJSON)
country_to_iso = {
    'Algeria': 'DZA',
    'Bahrain': 'BHR',
    'Egypt': 'EGY',
    'Iran': 'IRN',
    'Iraq': 'IRQ',
    'Israel': 'ISR',
    'Jordan': 'JOR',
    'Kuwait': 'KWT',
    'Lebanon': 'LBN',
    'Libya': 'LBY',
    'Morocco': 'MAR',
    'Oman': 'OMN',
    'Qatar': 'QAT',
    'Saudi Arabia': 'SAU',
    'Syria': 'SYR',
    'Tunisia': 'TUN',
    'United Arab Emirates': 'ARE',
    'Uae': 'ARE',  # in case your CSV uses this abbreviation
    'Yemen': 'YEM',
    'Palestine': 'PSE',
    'Palestinian Territories': 'PSE'
}

# Filter the DataFrame to include only rows with countries in our mapping
sales_df = sales_df[sales_df['Country'].isin(country_to_iso.keys())].copy()

# Convert the Sales column to numeric (forcing non-numeric to NaN)
sales_df['Sales'] = pd.to_numeric(sales_df['Sales'], errors='coerce')

# Map country names to their ISO codes and add as a new column
sales_df['ISO'] = sales_df['Country'].map(country_to_iso)
mena_iso_codes = sales_df['ISO'].dropna().unique().tolist()

# Load a GeoJSON file containing world country boundaries (the 'id' field here is the ISO3 code)
geojson_url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data/world-countries.json'
geo_data = requests.get(geojson_url).json()

# Filter the GeoJSON to include only features with IDs matching your MENA ISO codes
mena_geo = {
    "type": "FeatureCollection",
    "features": [feature for feature in geo_data["features"] if feature["id"] in mena_iso_codes]
}

# Enrich each GeoJSON feature with Sales and Agent data from the DataFrame,
# converting any NumPy numeric types to native Python types.
for feature in mena_geo["features"]:
    iso = feature["id"]
    matching = sales_df[sales_df["ISO"] == iso]
    if not matching.empty:
        sales_value = matching.iloc[0]["Sales"]
        # Convert the Sales value to a native Python float if it's not null
        feature["properties"]["Sales"] = float(sales_value) if pd.notnull(sales_value) else None
        # Check for an 'Agent' column; if not, check for 'Agent Name'
        if "Agent" in sales_df.columns:
            feature["properties"]["Agent"] = matching.iloc[0]["Agent"]
        elif "Agent Name" in sales_df.columns:
            feature["properties"]["Agent"] = matching.iloc[0]["Agent Name"]

# Create a Folium map centered on the MENA region
m = folium.Map(location=[26, 17], zoom_start=5, max_bounds=True)
# Fit the map to an approximate MENA bounding box (adjust these coordinates as needed)
m.fit_bounds([[10, -10], [40, 60]])

# Add the choropleth layer with the green color scale and clear black borders
folium.Choropleth(
    geo_data=mena_geo,
    name='choropleth',
    data=sales_df,
    columns=['ISO', 'Sales'],
    key_on='feature.id',
    fill_color='YlGn',       # Green color scale: lighter for lower sales, darker for higher
    fill_opacity=0.7,
    line_opacity=1,
    line_color='black',
    legend_name='Sales (in USD)'
).add_to(m)

# Create a tooltip that displays the country name, sales, and agent
tooltip = folium.GeoJsonTooltip(
    fields=['name', 'Sales', 'Agent'],
    aliases=['Country:', 'Sales (USD):', 'Agent:'],
    localize=True
)

# Add the enriched GeoJSON layer with the custom tooltip
folium.GeoJson(
    mena_geo,
    style_function=lambda feature: {
        'fillColor': 'transparent',
        'color': 'black',
        'weight': 2,
    },
    tooltip=tooltip
).add_to(m)

# Add a custom glossary box explaining the color scale
legend_html = '''
<div style="position: fixed;
     bottom: 50px; left: 50px; width: 250px; height: 120px;
     border:2px solid grey; z-index:9999; font-size:14px;
     background-color:white;
     padding: 10px;
     ">
     <b>Glossary</b><br>
     <i>Sales Scale:</i><br>
     Lighter green: Lower sales<br>
     Darker green: Higher sales<br>
     (Values in USD)
</div>
'''
m.get_root().html.add_child(folium.Element(legend_html))

# Display the interactive map
m


Saving 2024_sales_per_country.csv to 2024_sales_per_country (7).csv
Loaded 2024_sales_per_country (7).csv with shape: (7, 3)
Unique country names in data: ['Kuwait' 'Qatar' 'Saudi Arabia' 'Jordan' 'Bahrain'
 'Palestinian Territories' 'Oman']


# Table


In [29]:
<!DOCTYPE html>
<html>
<head>
  <!-- DataTables CSS -->
  <link rel="stylesheet" type="text/css" href="https://cdn.datatables.net/1.12.1/css/jquery.dataTables.css">
  <style>
    body { font-family: Arial, sans-serif; margin: 20px; }
    table.dataTable tbody tr:hover { background-color: #f1f1f1; }
    .toggle-cell { cursor: pointer; }
  </style>
</head>
<body>
  <table id="salesTable" class="display" style="width:100%">
    <thead>
      <tr>
        <th>Country</th>
        <th>Agent / Sales</th>
      </tr>
    </thead>
    <tbody>
      <!-- Data rows will be inserted here by Python -->
      <!-- Example row:
      <tr>
        <td>Egypt</td>
        <td class="toggle-cell" data-agent="John Doe" data-sales="15000">John Doe</td>
      </tr>
      -->
    </tbody>
  </table>

  <!-- jQuery and DataTables JS -->
  <script src="https://code.jquery.com/jquery-3.6.0.min.js"></script>
  <script type="text/javascript" charset="utf8" src="https://cdn.datatables.net/1.12.1/js/jquery.dataTables.js"></script>
  <script>
    $(document).ready(function() {
      // Initialize DataTables
      var table = $('#salesTable').DataTable({
        "paging": true,
        "searching": true,
        "info": true,
      });

      // Toggle between agent and sales on cell click
      $('#salesTable tbody').on('click', 'td.toggle-cell', function () {
          var cell = $(this);
          var currentText = cell.text();
          var agent = cell.data('agent').toString();
          var sales = cell.data('sales').toString();
          if (currentText === agent) {
              cell.text(sales);
              cell.css("background-color", "#d4edda"); // light green for sales view
          } else {
              cell.text(agent);
              cell.css("background-color", "#cce5ff"); // light blue for agent view
          }
      });
    });
  </script>
</body>
</html>


SyntaxError: invalid decimal literal (<ipython-input-29-3017ad637758>, line 7)

In [30]:
from IPython.display import HTML, display

html_content = """
<!DOCTYPE html>
<html>
<head>
  <link rel="stylesheet" type="text/css" href="https://cdn.datatables.net/1.12.1/css/jquery.dataTables.css">
  <style>
    body { font-family: Arial, sans-serif; margin: 20px; }
    table.dataTable tbody tr:hover { background-color: #f1f1f1; }
    .toggle-cell { cursor: pointer; background-color: #cce5ff; padding: 8px; }
  </style>
</head>
<body>
  <table id="salesTable" class="display" style="width:100%">
    <thead>
      <tr>
        <th>Country</th>
        <th>Agent / Sales</th>
      </tr>
    </thead>
    <tbody>
      <!-- Table rows will go here -->
      <tr>
        <td>Egypt</td>
        <td class="toggle-cell" data-agent="John Doe" data-sales="15000">John Doe</td>
      </tr>
      <tr>
        <td>Morocco</td>
        <td class="toggle-cell" data-agent="Jane Smith" data-sales="23000">Jane Smith</td>
      </tr>
    </tbody>
  </table>
  <script src="https://code.jquery.com/jquery-3.6.0.min.js"></script>
  <script type="text/javascript" charset="utf8" src="https://cdn.datatables.net/1.12.1/js/jquery.dataTables.js"></script>
  <script>
    $(document).ready(function() {
      var table = $('#salesTable').DataTable({
        "paging": true,
        "searching": true,
        "info": true,
      });
      $('#salesTable tbody').on('click', 'td.toggle-cell', function () {
          var cell = $(this);
          var currentText = cell.text();
          var agent = cell.data('agent').toString();
          var sales = cell.data('sales').toString();
          if (currentText === agent) {
              cell.text(sales);
              cell.css("background-color", "#d4edda"); // light green for sales view
          } else {
              cell.text(agent);
              cell.css("background-color", "#cce5ff"); // light blue for agent view
          }
      });
    });
  </script>
</body>
</html>
"""

display(HTML(html_content))


Country,Agent / Sales
Egypt,John Doe
Morocco,Jane Smith


In [31]:
from IPython.display import HTML, display
import io
import pandas as pd
from google.colab import files

# Upload and load the CSV file
uploaded = files.upload()
filename = list(uploaded.keys())[0]
df = pd.read_csv(io.BytesIO(uploaded[filename]))

# Standardize column names
df.columns = [col.strip().title() for col in df.columns]

# Determine the agent column
if 'Agent' in df.columns:
    agent_col = 'Agent'
elif 'Agent Name' in df.columns:
    agent_col = 'Agent Name'
else:
    raise ValueError("No agent column found in the data.")

# Build HTML rows for the table
rows = ""
for _, row in df.iterrows():
    country = row['Country']
    agent = row[agent_col]
    sales = row['Sales']
    sales_str = f"{sales}" if pd.notnull(sales) else "N/A"
    rows += f"""
    <tr>
      <td>{country}</td>
      <td class="toggle-cell" data-agent="{agent}" data-sales="{sales_str}">{agent}</td>
    </tr>
    """

# Build the full HTML content (using the DataTables example above)
html_content = f"""
<!DOCTYPE html>
<html>
<head>
  <link rel="stylesheet" type="text/css" href="https://cdn.datatables.net/1.12.1/css/jquery.dataTables.css">
  <style>
    body {{ font-family: Arial, sans-serif; margin: 20px; }}
    table.dataTable tbody tr:hover {{ background-color: #f1f1f1; }}
    .toggle-cell {{ cursor: pointer; background-color: #cce5ff; padding: 8px; }}
  </style>
</head>
<body>
  <table id="salesTable" class="display" style="width:100%">
    <thead>
      <tr>
        <th>Country</th>
        <th>Agent / Sales</th>
      </tr>
    </thead>
    <tbody>
      {rows}
    </tbody>
  </table>
  <script src="https://code.jquery.com/jquery-3.6.0.min.js"></script>
  <script type="text/javascript" charset="utf8" src="https://cdn.datatables.net/1.12.1/js/jquery.dataTables.js"></script>
  <script>
    $(document).ready(function() {{
      var table = $('#salesTable').DataTable({{
        "paging": true,
        "searching": true,
        "info": true,
      }});
      $('#salesTable tbody').on('click', 'td.toggle-cell', function () {{
          var cell = $(this);
          var currentText = cell.text();
          var agent = cell.data('agent').toString();
          var sales = cell.data('sales').toString();
          if (currentText === agent) {{
              cell.text(sales);
              cell.css("background-color", "#d4edda");
          }} else {{
              cell.text(agent);
              cell.css("background-color", "#cce5ff");
          }}
      }});
    }});
  </script>
</body>
</html>
"""

display(HTML(html_content))


Saving 2024_sales_per_country.csv to 2024_sales_per_country (9).csv


Country,Agent / Sales
Kuwait,Kuwait Protein
Qatar,ZAD
Saudi Arabia,Tamer
Jordan,CJ
Bahrain,ALJazira
Palestinian Territories,Sweetline
Oman,Catering S
